In [ ]:
import pandas as pd
import numpy as np
import cobra

cobra_config = cobra.Configuration()
cobra_config.solver = "gurobi"

BaseDir = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM_OB3b/'
ModelDir = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM_OB3b/model/'
TpmDir = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM_OB3b/transcriptomic/'

# Model analysis

## 1. pFBA and FVA solutions

In [ ]:
############## Define model files and names ##############
smmo = ModelDir + "GEM_ADVE_OB3b_sMMO.xml"
pmmo_ra = ModelDir + "GEM_ADVE_OB3b_pMMO_RA.xml"
pmmo_ut = ModelDir + "GEM_ADVE_OB3b_pMMO_UT.xml"
pmmo_dc = ModelDir + "GEM_ADVE_OB3b_pMMO_DC.xml"

model_files = [smmo, pmmo_ra, pmmo_ut, pmmo_dc]
model_names = ['sMMO', 'pMMO_ra', 'pMMO_ut', 'pMMO_dc']

############## Define pathways and functions for analysis ##############
### Reactions in sub-pathways
ch4_oxidation_via_thf   = ['rxn12217_c0','rxn00907_c0','rxn01211_c0','rxn00690_c0']
ch4_oxidation_via_h4mpt = ['rxn07847_c0','rxn07848_c0','rxn37236_c0','rxn02480_c0','rxn02431_c0','rxn07849_c0']
ser_cyc = ['rxn00424_c0','rxn01011_c0','rxn01013_c0','rxn01102_c0',
           'rxn01106_c0','rxn00459_c0','rxn00251_c0','rxn00248_c0']
tca_cyc = ['rxn00256_c0','rxn00974_c0','rxn01388_c0','rxn00198_c0','rxn08094_c0']
emc_cyc = ['rxn00178_c0','rxn01451_c0','rxn01453_c0','rxn02345_c0',
           'rxn02167_c0','rxn16150_c0','rxn16825_c0','rxn16151_c0','rxn25269_c0',
           'rxn03440_c0','rxn00682_c0','rxn01355_c0','rxn01996_c0','rxn00602_c0']


path_names = ['ch4_oxidation_via_thf', 'ch4_oxidation_via_h4mpt', 'serine_cyc', 'tca_cyc', 'emc_cyc']

paths = [ch4_oxidation_via_thf, ch4_oxidation_via_h4mpt, ser_cyc, tca_cyc, emc_cyc]

cycles = [ser_cyc, tca_cyc, emc_cyc]


In [ ]:
pfba_sol_names = ['sMMO_pFBA.csv', 'pMMO_RA_pFBA.csv', 'pMMO_UT_pFBA.csv', 'pMMO_DC_pFBA.csv']
pfba_list = []
normalized_pfba_list = []
obj = 'bio1_biomass'

for file in model_files:
    md = cobra.io.read_sbml_model(file)

    md.reactions.EX_cpd00007_e0.bounds = (-21.2,0)
    md.reactions.EX_cpd01024_e0.bounds = (-1000,0)
    md.reactions.NGAM_c0.bounds = (25.3,25.3)

    

#     # If testing RuMP
#     rump = cobra.Reaction("RuMP")
#     rump.name = 'RuMP'
#     rump.lower_bound = 0
#     rump.upper_bound = 1000
#     rump.add_metabolites({
#         md.metabolites.get_by_id('cpd00055_c0'): -1.0,
#         md.metabolites.get_by_id('cpd00171_c0'): -1.0,

#         md.metabolites.get_by_id('cpd00072_c0'): 1.0
#     })
#     md.add_reactions([rump])
    
    

#     # If testing ICL
#     icl = cobra.Reaction("ICL")
#     icl.name = 'ICL'
#     icl.lower_bound = 0
#     icl.upper_bound = 1000
#     icl.add_metabolites({
#         md.metabolites.get_by_id('cpd00260_c0'): -1.0,

#         md.metabolites.get_by_id('cpd00036_c0'): 1.0,
#         md.metabolites.get_by_id('cpd00040_c0'): 1.0
#     })
#     md.add_reactions([icl])



    md.objective = obj
    
    ## Original solutions
    md_pfba = cobra.flux_analysis.pfba(md)
    pfba_list.append(md_pfba)

    ## Normalized to assimilated methane flux to investigate fluxes in central metabolic pathways
    normalized_pfba_sol = md_pfba.fluxes/md_pfba.fluxes.loc['rxn00692_c0']
    normalized_pfba_list.append(normalized_pfba_sol)    
    
for sol, norm_sol, name in zip(pfba_list, normalized_pfba_list, pfba_sol_names):
    
    sol.fluxes.to_csv(BaseDir + 'analysis/pfba_sol/' + name)    
    norm_sol.to_csv(BaseDir + 'analysis/pfba_sol/' + 'normalized_' + name)

#     # If testing RuMP
#     sol.fluxes.to_csv('RuMP_' + name)    
#     norm_sol.to_csv('normalized_RuMP_' + name)

#     # If testing ICL
#     sol.fluxes.to_csv('ICL_' + name)    
#     norm_sol.to_csv('normalized_ICL_' + name)

In [ ]:
########## FVA ##########
fva_sol_names = ['sMMO_fva.csv', 'pMMO_RA_fva.csv', 'pMMO_UT_fva.csv', 'pMMO_DC_fva.csv']
fva_list  = []
obj = 'bio1_biomass'

for file, fva_name in zip(model_files, fva_sol_names):
    
    md = cobra.io.read_sbml_model(file)
    
    md.reactions.EX_cpd00007_e0.bounds = (-21.2,0)
    md.reactions.EX_cpd01024_e0.bounds = (-1000,0)
    md.reactions.NGAM_c0.bounds = (25.3,25.3)

    
    
#     # If testing RuMP
#     rump = cobra.Reaction("RuMP")
#     rump.name = 'RuMP'
#     rump.lower_bound = 0
#     rump.upper_bound = 1000
#     rump.add_metabolites({
#         md.metabolites.get_by_id('cpd00055_c0'): -1.0,
#         md.metabolites.get_by_id('cpd00171_c0'): -1.0,

#         md.metabolites.get_by_id('cpd00072_c0'): 1.0
#     })
#     md.add_reactions([rump])



#     # If testing ICL
#     icl = cobra.Reaction("ICL")
#     icl.name = 'ICL'
#     icl.lower_bound = 0
#     icl.upper_bound = 1000
#     icl.add_metabolites({
#         md.metabolites.get_by_id('cpd00260_c0'): -1.0,

#         md.metabolites.get_by_id('cpd00036_c0'): 1.0,
#         md.metabolites.get_by_id('cpd00040_c0'): 1.0
#     })
#     md.add_reactions([icl])    
    
    
    
    md.objective = obj

    md_fva  = cobra.flux_analysis.flux_variability_analysis(md)
    
    md_fva.to_csv(BaseDir + 'analysis/fva_sol/' + fva_name)
    
#     # If testing RuMP
#     md_fva.to_csv(BaseDir + 'analysis/pfba_sol/' + 'RuMP_' + fva_name)
    
#     # If testing ICL
#     md_fva.to_csv(BaseDir + 'analysis/pfba_sol/' + 'ICL_' + fva_name)
    
    fva_list.append(md_fva)

## 2. solution analysis

In [ ]:
### Oxygen/methane uptake rate ratio and assimilation percentage in naive model

df_index = [
    # 'Minimum required methane consumption', 'Minimum required oxygen consumption',
    'Growth rate', 'O2 consumption flux','CH4 consumption flux', 'O2/CH4 ratio', 'CH4 assmilation %', 
    'CO2 assimilation %', 'CO2_EMC/total', 'CO2_PEP/total',
    'CH4/NO3 assimilation ratio',
    'emc/serine', 'tca/serine', 'glyoxylate_regen'
]

df_dict = {}

for pfba_sol, file, name in zip(pfba_list, model_files, model_names):
    
    df_value = []
    
#     md = cobra.io.read_sbml_model(file)
#     md.reactions.EX_cpd00007_e0.bounds = (-100,0)
#     md.reactions.EX_cpd01024_e0.bounds = (-1000,0)
    
#     if name == 'sMMO':
#         obj = 'EX_cpd01024_e0'
#         md.reactions.bio1_biomass.bounds = (0.1, 0.1)
#         md.objective = obj

#         md_pfba = cobra.flux_analysis.pfba(md)

#         df_value.append(abs(md_pfba.fluxes.EX_cpd01024_e0))
#         df_value.append(abs(md_pfba.fluxes.EX_cpd00007_e0))
    
#     else:
#         obj = 'EX_cpd01024_e0'
#         md.reactions.bio1_biomass.bounds = (0.12, 0.12)
#         md.objective = obj

#         md_pfba = cobra.flux_analysis.pfba(md)

#         df_value.append(abs(md_pfba.fluxes.EX_cpd01024_e0))
#         df_value.append(abs(md_pfba.fluxes.EX_cpd00007_e0))
    
    
#     md = cobra.io.read_sbml_model(file)
#     md.reactions.EX_cpd00007_e0.bounds = (-21.2,0)
#     md.reactions.EX_cpd01024_e0.bounds = (-1000,0)        
    
    
    df_value.append(abs((pfba_sol.fluxes.bio1_biomass)))
    df_value.append(abs((pfba_sol.fluxes.EX_cpd00007_e0)))    
    df_value.append(abs((pfba_sol.fluxes.EX_cpd01024_e0)))

    df_value.append(abs((pfba_sol.fluxes.EX_cpd00007_e0)/(pfba_sol.fluxes.EX_cpd01024_e0)))
    df_value.append(abs((pfba_sol.fluxes.rxn00692_c0)*100/(pfba_sol.fluxes.EX_cpd01024_e0)))
    

    co2_summary = md.metabolites.get_by_id('cpd00011_c0').summary(solution = pfba_sol).consuming_flux.sort_values(by = ['percent'], ascending = False)
    assmilated_co2_flux = (abs(sum(co2_summary.loc[:, 'flux'].values))-abs(co2_summary.loc['rxn05467_c0', 'flux']))
    co2_assimilation_percent = 100*assmilated_co2_flux/(assmilated_co2_flux+abs(pfba_sol.fluxes.rxn00692_c0))
    df_value.append(co2_assimilation_percent)
    df_value.append((abs(pfba_sol.fluxes.rxn16150_c0)+abs(pfba_sol.fluxes.rxn01355_c0))/assmilated_co2_flux)
    df_value.append(abs(pfba_sol.fluxes.rxn00251_c0)/assmilated_co2_flux)



    df_value.append(abs((pfba_sol.fluxes.rxn00692_c0)/(pfba_sol.fluxes.EX_cpd00209_e0)))

#     afa_cyc_list = []
#     for cyc in cycles:
#         afa_cyc = 0
#         for rxn in cyc:
#             afa_cyc += abs(pfba_sol.fluxes.loc[rxn])
#         afa_cyc = afa_cyc/len(cyc)
#         afa_cyc_list.append(afa_cyc)

#     df_value.append(afa_cyc_list[2]/afa_cyc_list[0])
#     df_value.append(afa_cyc_list[1]/afa_cyc_list[0])
    
    df_value.append(abs((pfba_sol.fluxes.rxn00178_c0)/(pfba_sol.fluxes.rxn00692_c0)))
    df_value.append(abs((pfba_sol.fluxes.rxn00974_c0)/(pfba_sol.fluxes.rxn00692_c0)))
    df_value.append(abs((pfba_sol.fluxes.rxn00682_c0)/(pfba_sol.fluxes.rxn00692_c0)))

    df_dict[name] = df_value

df_naive_model = pd.DataFrame(df_dict, index = df_index)


# print('O2 uptake fva:        ', rxn_fva_deviation('EX_cpd00007_e0', wt_pfba, wt_fva))
# print('CH4 uptake fva:       ', rxn_fva_deviation('EX_cpd01024_e0', wt_pfba, wt_fva))
# print('CH4 assimilation fva: ', rxn_fva_deviation('rxn00692_c0', wt_pfba, wt_fva))
# print('CH4 oxidation fva:    ', rxn_fva_deviation('rxn00371_c0', wt_pfba, wt_fva))

df_naive_model
